###### MobileNet: Simple CNN model used in mobile application, faster, : depth-wise seperable convolution

### load libraries

In [1]:
from flask import Flask, request, jsonify, url_for, render_template
import uuid #unique id so that it can be used as a identifier for predictions
import os

In [2]:
Allowed_extension = set(["png", "jpg", "jpeg", "gif" ])

In [3]:
def allowed_file(filename):
    return "." in filename and filename.rsplit(".")[1] in Allowed_extension

## create flask application

In [4]:
app = Flask(__name__) #object : resources: html, css, js and other files

In [5]:
@app.route('/')
def index():
    return render_template('ImageML.html')

# create model

In [6]:
import numpy as np
from werkzeug.utils import secure_filename

#CNN mobileNet model
from PIL import Image, ImageFile

from io import BytesIO

In [7]:
#model 
from tensorflow.keras.applications import MobileNet  #cnn model

#preprocessing libraries
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications.mobilenet import decode_predictions

In [8]:
model = MobileNet(weights = "imagenet", include_top = True)

In [9]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [10]:
#API: waiter -> order food and he/she brings your food

In [11]:
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
IMAGE_CHANNEL = 3


In [12]:
@app.route('/api/image', methods = ["POST"])
def upload_image():
    if 'image' not in request.files:
        return render_template("ImageML.html", prediction = "No Posted Image")
    file = request.files["image"] #else part 
    
    if file.filename == '':
        return render_template("ImageML.html", prediction = "You did not select an image")
    
    #correcly solved then we say that file is sucessful to be uploaded
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        
        x = [] # numpy array containing pixels of image
        ImageFile.LOAD_TRUNCATED_IMAGES = False
        
        #load image here
        img = Image.open(BytesIO(file.read()))
        img.load()
        
        #This is all to be done for model
        img = img.resize((IMAGE_WIDTH, IMAGE_HEIGHT), Image.ANTIALIAS)
        
        #numpy array
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis = 0)
        
        x = preprocess_input(x) #input is ready
        
        pred = model.predict(x)
        lst = decode_predictions(pred, top = 3 )
        
        items = []
        for item in lst[0]:
            items.append({"name": item[1], 'prob': float(item[2])})
            
        response = {'pred': items}
        return render_template('ImageML.html', prediction = f'My Prediction: {response["pred"]}')
    
    else:
        return render_template("ImageML.html", prediction = "Invalid file extension")

# running our code

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader = False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Oct/2022 18:24:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2022 18:24:57] "GET /static/css/style.css HTTP/1.1" 304 -


1/1 [==============================] - 0s 379ms/step


127.0.0.1 - - [25/Oct/2022 18:25:03] "POST /api/image HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2022 18:25:03] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Oct/2022 18:25:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2022 18:25:18] "GET /static/css/style.css HTTP/1.1" 304 -


1/1 [==============================] - 0s 41ms/step


127.0.0.1 - - [25/Oct/2022 18:25:22] "POST /api/image HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2022 18:25:22] "GET /static/css/style.css HTTP/1.1" 304 -
